# Leaderboards
https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
https://llm-leaderboard.streamlit.app/
https://opening-up-chatgpt.github.io/
https://www.linkedin.com/search/results/content/?keywords=rag%20zephyr&searchId=63f84b12-44fc-403e-971b-7b0f14c12dc6&sid=aP%40&update=urn%3Ali%3Afs_updateV2%3A(urn%3Ali%3Aactivity%3A7132380266773250049%2CBLENDED_SEARCH_FEED%2CEMPTY%2CDEFAULT%2Cfalse)

# Llama2 
https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

## Not really open source:
Can't be used to train other models: 
“v. You will not use the Llama Materials or any output or results of the Llama Materials to improve any other large language model (excluding Llama 2 or derivative works thereof).”
Commercial restrictions:
“2. Additional Commercial Terms. If, on the Llama 2 version release date, the monthly active users of the products or services made available by or for Licensee, or Licensee’s affiliates, is greater than 700 million monthly active users in the preceding calendar month, you must request a license from Meta,“
Not approved by the [Open Source Initiative]("https://opensource.org/licenses/")

## BUT: 
huggingface leaderboard is dominated by llama 2 models
Open Technology to some degree

# Inactive/Discontinued: 
Open Assistant
BlOOMChat
Pythia-Chat_base

# Mistral 7B
Nach eigener Aussage: "No, it is free to use but not open source. The datasets and weights are proprietary."
*Aber* Apache 2.0 license -> Mehr Open Source 
Zephyr finetuned und bessere Performance, aber auch als Mistral-7B-Instruct?; https://huggingface.co/HuggingFaceH4/zephyr-7b-beta
Kein Multilinguales Model. Ein Hugginface User finetuned german modelle; https://huggingface.co/flozi00
Ansonsten translation?

# Conclustions
We will be swapping models more regularly with open source


In [4]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ and/or you do not have an MPS-enabled device on this machine.")

In [ ]:
# zephyr
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.float32, device_map="auto") 
# torch_dtype=torch.bfloat16; BFloat16 is not supported on MPS

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
# Error: still using torch.bfloat16
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

In [ ]:

# Mistral
device = "mps"

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(encodeds, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

# Out of memory error
# There are some https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF
# and https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF
# models that are quantized and use less memory, but use GGUF and llama.cpp, but does that support mps?

# Embedding
MTEB Benchmark: https://huggingface.co/blog/mteb, https://www.youtube.com/watch?v=Tilgr1hjziw
Leaderboards: https://huggingface.co/spaces/mteb/leaderboard
Metric used for retrieval evaluation: Normalized Discounted Cumulative Gain. Explained: https://www.youtube.com/watch?v=BvRMAgx0mvA
!Only tested for english texts
?Are datasets chat relevant?

# Models
voyage - not open source
cohere - not open source
BAAI/bge - no german/multilinugal
gte-large - no german/multilingual

Next best: multilingual-e5-large https://huggingface.co/intfloat/multilingual-e5-large

In [5]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

[[90.81393432617188, 72.13392639160156], [70.53543090820312, 88.76109313964844]]
